## Baseline_tables

#### Import libraries

In [1]:
# Step 1: Import libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
from sqlalchemy import text
from sqlalchemy import create_engine
from shapely.geometry import LineString

C:\Users\monaliza\Miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Create the connection with the DB (optional)

In [2]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [3]:
# Step 2: Create a new schema when it is necessary
# schema = '''CREATE SCHEMA name_schema'''
# conn.execute(schema)

### Nodes

###### To create the nodes table, it is necessary to merge OSM (places and streets) data with the GTFS data (stops).

#### Places baseline table

In [4]:
# Step 1: Extract the geographic information from the OSM database.
# In this example, we are utilizing data from Västra Götaland, although these can be modified based on specific requirements.
places = ox.geocode_to_gdf(['Askersund, Sweden','Ale, Sweden','Alingsås, Sweden',
                            'Åmål, Sweden', 'Bengtsfors, Sweden', 'Bollebygd, Sweden',
                            ' Borås, Sweden', 'Dals-Ed, Sweden','Essunga, Sweden',
                            'Falköping, Sweden','Färgelanda, Sweden', 'Göteborg, Sweden',
                            'Götene, Sweden', 'Grästorp, Sweden', 'Gullspång, Sweden',
                            'Härryda, Sweden', 'Herrljunga, Sweden', 'Hjo, Sweden',
                            'Karlsborg, Sweden', 'Kungälv, Sweden', 'Lerum, Sweden',
                            'Lidköping, Sweden', 'Lilla Edet, Sweden', 'Lysekil, Sweden',
                            'Mariestad, Sweden', 'Mark, Sweden', 'Mellerud, Sweden',
                            'Mölndal, Sweden', 'Munkedal, Sweden', 'Öckerö, Sweden',
                            'Orust, Sweden','Partille, Sweden','Skara, Sweden',
                            'Skövde, Sweden','Sotenäs, Sweden','Stenungsund, Sweden',
                            'Strömstad, Sweden', 'Svenljunga, Sweden', 'Tanum, Sweden',
                            'Tibro, Sweden', 'Tidaholm, Sweden', 'Tjörn, Sweden',
                            'Töreboda, Sweden', 'Tranemo, Sweden','Trollhättan, Sweden',
                            'Uddevalla, Sweden', 'Ulricehamn, Sweden', 'Vänersborg, Sweden',
                            'Vara, Sweden','Vårgårda, Sweden', 'Halland, Sweden',
                            'Jonkoping, Sweden', 'Ostergotland, Sweden', 'Orebro, Sweden',
                            'Varmland, Sweden', 'Habo, Sweden', 'Hallsberg,Sweden',
                            'Lekeberg,Sweden', 'Mullsjö, Sweden', 'Gislaved,Sweden',
                            'Karlsborg, Sweden', 'Laxa, Sweden', 'Varberg, Sweden',
                            'Nässjö, Sweden', 'Gnösjö, Sweden', 'Falkenberg, Sweden',
                            'Vaggeryd, Sweden', 'Värnamo, Sweden', 'Halden, Norway',
                            'Kumla, Sweden'
                            ])

In [5]:
# Step 2: Reproject the places GeoDataFrame according to specific requirements.
places = ox.projection.project_gdf(to_crs='epsg:3006', gdf= places)

# Step 3 (optional): Display the places GeoDataFrame
places.head()

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"MULTIPOLYGON (((478344.395 6521271.033, 478348...",59.026161,58.646225,15.347592,14.624906,143326451,relation,935513,58.866667,14.916667,boundary,administrative,14,0.450112,municipality,Askersunds kommun,"Askersunds kommun, Örebro County, Sweden"
1,"POLYGON ((322429.991 6416232.985, 322432.839 6...",58.118035,57.806263,12.434704,12.007589,146629155,relation,935506,58.000000,12.283333,boundary,administrative,14,0.471351,municipality,Ale kommun,"Ale kommun, Västra Götaland County, Sweden"
2,"POLYGON ((342627.196 6426078.884, 342647.946 6...",58.198126,57.761922,12.710975,12.340988,145826287,relation,935547,57.999850,12.500037,boundary,administrative,14,0.469773,municipality,Alingsås kommun,"Alingsås kommun, Västra Götaland County, Sweden"
3,"POLYGON ((350801.238 6543563.250, 351188.303 6...",59.186071,58.747233,12.899666,12.402254,143743192,relation,935601,58.983838,12.612671,boundary,administrative,14,0.462139,municipality,Åmåls kommun,"Åmåls kommun, Västra Götaland County, Sweden"
4,"MULTIPOLYGON (((319014.499 6567146.864, 319302...",59.263584,58.749612,12.468416,11.830215,143763211,relation,935491,59.032659,12.219105,boundary,administrative,14,0.473632,municipality,Bengtsfors kommun,"Bengtsfors kommun, Västra Götaland County, Sweden"


In [6]:
# Step 4 (optional): Export the file to the DB
places.to_postgis('places', engine, schema='pt_0_baseline', if_exists='replace')

#### Streets baseline table

In [7]:
# Step 1: Create the street baseline table
# In this example, we are importing an extracted and cleaned OSM street network model from our database. To see an example of how to do this, please refer to the specific folder.
# This street network model can also originate from other sources, provided the streets have unique IDs.
street_network_query = text('SELECT * FROM p2_gm_opensourcedata.p2_gm_allnetwork')
street_network = gpd.read_postgis(street_network_query, engine, geom_col='geometry')

#### Create the nodes model

In [8]:
# Step 1: Import the necessary files from the GTFS dataset (stops.txt)
# In this example, we are utilizing data from Västra Götaland, although these can be modified based on specific requirements.
#stops
raw_stops_gtfs = text('''SELECT * FROM p1_gtfs.regional_stops''')
stops_gtfs = gpd.read_postgis(raw_stops_gtfs, engine, geom_col='geometry')

In [9]:
# Preprocess the data
# Step 3: Create the stops_gtfs GeoDataFrame from the oiginal file
stops_geodataframe = gpd.GeoDataFrame(stops_gtfs, geometry=gpd.points_from_xy(stops_gtfs.stop_lon,
                                                                              stops_gtfs.stop_lat), crs='epsg:4326')

# Step 4: Reproject the GeoDataFrame according to specific requirements.
stops_geodataframe = ox.projection.project_gdf(stops_geodataframe, to_crs='epsg:3006')

In [10]:
# Join with the places table
# Step 5: Create the spatial join between the 'stops_geodataframe' and 'places' GeoDataFrames
nodes_sjoin_l1 = stops_geodataframe.sjoin (places, how="left")

# Step 6: Set the geometry of the 'stop_place_id' GeoDataFrame
nodes_sjoin_l1 = gpd.GeoDataFrame(nodes_sjoin_l1, geometry='geometry', crs='EPSG:3006')

# Step 7: Clean the data by removing 'fake' parent stations and dropping duplicate stop_IDs
nodes_sjoin_l1 = nodes_sjoin_l1 [nodes_sjoin_l1['parent_station'].notnull()].drop_duplicates(subset=['stop_id'])

# Step 8: Clean the data
nodes_sjoin_l1 = nodes_sjoin_l1 [['stop_id', 'stop_name', 'place_id', 'geometry']]

# Step 9 (optional): Display the 'nodes_baseline' GeoDataFrame
nodes_sjoin_l1.head()

,stop_id,stop_name,place_id,geometry
9236,9022014001004001,Amhult Resecentrum,145235353,POINT (308137.973 6400982.960)
9237,9022014001004002,Amhult Resecentrum,145235353,POINT (308174.952 6400959.904)
9238,9022014001004003,Amhult Resecentrum,145235353,POINT (308179.982 6400938.925)
9239,9022014001004004,Amhult Resecentrum,145235353,POINT (308152.980 6400960.949)
9240,9022014001004005,Amhult Resecentrum,145235353,POINT (308122.947 6400966.949)


In [11]:
# Join with the street network
# Step 10: Perform a spatial join between the 'sjoin_stops_places' and the 'street_network' GeoDataFrames
nodes_sjoin_l2 = nodes_sjoin_l1.sjoin_nearest(street_network, how='left',
                                              max_distance=500).drop_duplicates(subset=['stop_id','osmid'])

# Step 11: Clean the data
nodes_pt_0 = nodes_sjoin_l2 [['stop_id', 'stop_name', 'place_id', 'osmid', 'geometry']]

#Step 12 (optional): Display the 'nodes_baseline' GeoDataFrame
nodes_pt_0.head()

,stop_id,stop_name,place_id,osmid,geometry
9236,9022014001004001,Amhult Resecentrum,145235353,290390139.0,POINT (308137.973 6400982.960)
9237,9022014001004002,Amhult Resecentrum,145235353,846315401.0,POINT (308174.952 6400959.904)
9238,9022014001004003,Amhult Resecentrum,145235353,846315401.0,POINT (308179.982 6400938.925)
9239,9022014001004004,Amhult Resecentrum,145235353,846315401.0,POINT (308152.980 6400960.949)
9240,9022014001004005,Amhult Resecentrum,145235353,868843476.0,POINT (308122.947 6400966.949)


In [12]:
#Step 13 (optional): Export the 'nodes_baseline' GeoDataFrame to the DB
nodes_pt_0.to_postgis('nodes_pt_0', engine, schema="pt_0_baseline", if_exists ='replace')

### Links

In [13]:
# Step 1: Import the necessary files from the GTFS dataset
# In this example, we are utilizing data from Västra Götaland, although these can be modified based on specific requirements.
# routes
raw_routes_gtfs = text('''SELECT * FROM p1_gtfs.regional_routes''')
routes_gtfs = pd.read_sql(raw_routes_gtfs, engine)
# trips
raw_trips_gtfs = text('''SELECT * FROM p1_gtfs.regional_trips''')
trips_gtfs = pd.read_sql(raw_trips_gtfs, engine)
# stop_times
raw_stop_times_gtfs = text('''SELECT * FROM p1_gtfs.regional_stoptimes''')
stop_times_gtfs = pd.read_sql(raw_stop_times_gtfs, engine)

In [14]:
# Preprocess the data
# Step 2: Merge the gtfs tables
gtfs_merge_l1 = pd.merge(routes_gtfs [['route_id', 'route_short_name', 'route_type']],
                              trips_gtfs[['route_id', 'service_id', 'trip_id', 'direction_id']], on='route_id', how='inner')
gtfs_merge_l2 = pd.merge(gtfs_merge_l1, stop_times_gtfs[['trip_id', 'arrival_time', 'stop_id', 'stop_sequence']],
                                        on='trip_id', how='inner')

# Step 3: Fix DataFrame column types
gtfs_merge_l2['route_id'] = gtfs_merge_l2['route_id'].astype(str)
gtfs_merge_l2['trip_id'] = gtfs_merge_l2['trip_id'].astype(str)
gtfs_merge_l2['arrival_time'] = pd.to_timedelta(gtfs_merge_l2['arrival_time'])
gtfs_merge_l2['stop_id'] = gtfs_merge_l2['stop_id'].astype(str)

# Step 4: Set names for route types
conditions_routetype = [(gtfs_merge_l2['route_type'] == 100), (gtfs_merge_l2['route_type'] == 700),
                        (gtfs_merge_l2['route_type'] == 900), (gtfs_merge_l2['route_type'] == 1000),
                        (gtfs_merge_l2['route_type'] == 1501)]
values_routetype = ['railway_service', 'bus_service', 'tram_service', 'water_transport_service', 'communal_taxi_service']
gtfs_merge_l2 ['mode'] = np.select(conditions_routetype, values_routetype)

# Step 5: Set time periods based on the arrival_time
conditions_timeperiod = [(gtfs_merge_l2 ['arrival_time'] >'0 days 00:00:00') & (gtfs_merge_l2['arrival_time'] < '0 days 06:00:00'),
                         (gtfs_merge_l2['arrival_time'] >= '0 days 06:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 09:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 09:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 12:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 12:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 15:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 15:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 18:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 18:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 21:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 21:00:00') & (gtfs_merge_l2 ['arrival_time'] <= '0 days 24:00:00'),
                         (gtfs_merge_l2 ['arrival_time'] >= '0 days 24:00:00') & (gtfs_merge_l2 ['arrival_time'] > '1 days 00:00:00')]
values_timeperiod = ['night_dawn', 'peak_morning', 'morning', 'afternoon', 'peak_evening', 'evening', 'night','night_dawn']
gtfs_merge_l2 ['time_period'] = np.select(conditions_timeperiod, values_timeperiod)

# Step 6: Merge the DataFrame with the 'nodes_baseline' GeoDataFrame
links_pt_0_l1 = pd.merge(gtfs_merge_l2, nodes_pt_0[['stop_id', 'stop_name', 'place_id', 'geometry']], on='stop_id', how='inner')

# Step 7: Change the columns name
links_pt_0_l1.rename(columns={'stop_id': 'stop_id_source', 'stop_name': 'stop_name_source', 'place_id':'place_id_source' }, inplace = True)

# Step 8 (optional): display the 'links_baseline_l1' GeoDataFrame
links_pt_0_l1.head()

,route_id,route_short_name,route_type,service_id,trip_id,direction_id,arrival_time,stop_id_source,stop_sequence,mode,time_period,stop_name_source,place_id_source,geometry
0,9011092020200000,FLYG,700,360,141010001168286532,1,0 days 04:20:00,9022014004940004,1,bus_service,night_dawn,Nils Ericson Terminalen,145235353,POINT (319600.955 6400417.896)
1,9011092020200000,FLYG,700,360,141010001168286532,1,0 days 04:24:00,9022014004090004,2,bus_service,night_dawn,Kungsportsplatsen,145235353,POINT (319442.995 6399774.900)
2,9011092020200000,FLYG,700,360,141010001168286532,1,0 days 04:27:00,9022014001420002,3,bus_service,night_dawn,Berzeliigatan,145235353,POINT (320123.952 6399161.990)
3,9011092020200000,FLYG,700,360,141010001168286532,1,0 days 04:30:00,9022014003980014,4,bus_service,night_dawn,Korsvägen,145235353,POINT (320438.954 6398935.924)
4,9011092020200000,FLYG,700,360,141010001168286532,1,0 days 04:51:43,9022014018328002,5,bus_service,night_dawn,Hangarvägen,145582383,POINT (339142.962 6395582.971)


In [15]:
# Create the links between the stops
# Step 9: Create a copy of links_baseline_l1 to not use the original table
links_pt_0_l1_copy = links_pt_0_l1.copy()

# Step 10: Define a function to process each group within the groupby operation
def process_group(group):
    sorted_group = group.sort_values('stop_sequence')
    new_time = sorted_group['arrival_time'].shift(-1)

    return sorted_group.assign(
        new_time=pd.to_timedelta(new_time),
        time_distance=(new_time - sorted_group['arrival_time']).dt.total_seconds().astype(float),
        stop_id_target=sorted_group['stop_id_source'].shift(-1).astype(str),
        stop_name_target=sorted_group['stop_name_source'].shift(-1).astype(str),
        place_id_target=sorted_group['place_id_source'].shift(-1).fillna(0).astype(int),
        geometry=[
            LineString([[a.coords[0][0], a.coords[0][1]], [b.coords[0][0], b.coords[0][1]]]) if b else None
            for a, b in zip(sorted_group['geometry'], sorted_group['geometry'].shift(-1))
        ]
    )

# Step 11: Apply the processing function to each group using groupby
links_pt_0_l1 = (
    links_pt_0_l1_copy.groupby('trip_id')
    .apply(process_group)
    .reset_index(drop=True)
)

# Step 12 (optional): Display the 'links_baseline_l1' DataFrame (and check the links)
links_pt_0_l1.head()

C:\Users\monaliza\AppData\Local\Temp\ipykernel_172\174838690.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  links_pt_0_l1_copy.groupby('trip_id')


,route_id,route_short_name,route_type,service_id,trip_id,direction_id,arrival_time,stop_id_source,stop_sequence,mode,time_period,stop_name_source,place_id_source,geometry,new_time,time_distance,stop_id_target,stop_name_target,place_id_target
0,9011014505000000,50,700,5,141010001054458148,0,0 days 05:35:00,9022014001760012,1,bus_service,night_dawn,Brunnsparken,145235353,LINESTRING (319300.9681938812 6400118.94329882...,0 days 05:36:37,97.0,9022014006242004,Stenpiren,145235353
1,9011014505000000,50,700,5,141010001054458148,0,0 days 05:36:37,9022014006242004,2,bus_service,night_dawn,Stenpiren,145235353,LINESTRING (318713.9534298249 6400020.97166797...,0 days 05:39:40,183.0,9022014003645002,Järnvågen,145235353
2,9011014505000000,50,700,5,141010001054458148,0,0 days 05:39:40,9022014003645002,3,bus_service,night_dawn,Järnvågen,145235353,LINESTRING (318203.9588559234 6399558.95525878...,0 days 05:48:00,500.0,9022014001410002,Beryllgatan,145235353
3,9011014505000000,50,700,5,141010001054458148,0,0 days 05:48:00,9022014001410002,4,bus_service,night_dawn,Beryllgatan,145235353,LINESTRING (314286.9680952884 6395003.92383626...,0 days 05:50:06,126.0,9022014001370002,Bergkristallsgatan,145235353
4,9011014505000000,50,700,5,141010001054458148,0,0 days 05:50:06,9022014001370002,5,bus_service,night_dawn,Bergkristallsgatan,145235353,LINESTRING (314841.95194952097 6394771.9803292...,0 days 05:51:19,73.0,9022014006070002,Smyckegatan,145235353


In [16]:
# Clean the Data
# Step 13: Set source and target columns
links_pt_0 = links_pt_0_l1.rename(columns={'stop_id_source':'source', 'stop_id_target':'target'})

# Step 14: Filter columns
links_pt_0 = links_pt_0[['source', 'target', 'route_short_name', 'mode', 'time_distance',
                         'time_period', 'route_id','service_id', 'trip_id', 'direction_id',
                         'stop_sequence','stop_name_source', 'stop_name_target', 'place_id_source',
                         'place_id_target', 'geometry']]

# Step 15: Filter invalid geometries
links_pt_0 = links_pt_0[links_pt_0['source'] != links_pt_0['target']]

# Step 16: Drop None geometries
links_pt_0 = links_pt_0.dropna(subset=['geometry'])

# Step 17: Transform the DataFrame into a GeoDataFrame
links_pt_0 = gpd.GeoDataFrame(links_pt_0, geometry='geometry', crs='EPSG:3006')

# Step 18 (optional): Display the 'links_baseline' GeoDataFrame
links_pt_0.head()

,source,target,route_short_name,mode,time_distance,time_period,route_id,service_id,trip_id,direction_id,stop_sequence,stop_name_source,stop_name_target,place_id_source,place_id_target,geometry
0,9022014001760012,9022014006242004,50,bus_service,97.0,night_dawn,9011014505000000,5,141010001054458148,0,1,Brunnsparken,Stenpiren,145235353,145235353,"LINESTRING (319300.968 6400118.943, 318713.953..."
1,9022014006242004,9022014003645002,50,bus_service,183.0,night_dawn,9011014505000000,5,141010001054458148,0,2,Stenpiren,Järnvågen,145235353,145235353,"LINESTRING (318713.953 6400020.972, 318203.959..."
2,9022014003645002,9022014001410002,50,bus_service,500.0,night_dawn,9011014505000000,5,141010001054458148,0,3,Järnvågen,Beryllgatan,145235353,145235353,"LINESTRING (318203.959 6399558.955, 314286.968..."
3,9022014001410002,9022014001370002,50,bus_service,126.0,night_dawn,9011014505000000,5,141010001054458148,0,4,Beryllgatan,Bergkristallsgatan,145235353,145235353,"LINESTRING (314286.968 6395003.924, 314841.952..."
4,9022014001370002,9022014006070002,50,bus_service,73.0,night_dawn,9011014505000000,5,141010001054458148,0,5,Bergkristallsgatan,Smyckegatan,145235353,145235353,"LINESTRING (314841.952 6394771.980, 315135.959..."


In [17]:
# Step 19 (optional): Export the 'links_baseline' GeoDataFrame to the DB
links_pt_0.to_postgis('links_pt_0', engine, schema='pt_0_baseline', if_exists ='replace')